In [1]:
DATE_TASK = "2022-02-25-pseudomonas-phylogeny-AA"
DIR = mkpath("$(homedir())/workspace/$DATE_TASK")
cd(DIR)
DATE, TASK = match(r"^(\d{4}-\d{2}-\d{2})-(.*)$", DATE_TASK).captures

2-element Vector{Union{Nothing, SubString{String}}}:
 "2022-02-25"
 "pseudomonas-phylogeny-AA"

In [2]:
import Pkg
Pkg.update()
pkgs = [
"JSON",
"HTTP",
"Dates",
"uCSV",
"DelimitedFiles",
"DataFrames",
"ProgressMeter",
"BioSequences",
"FASTX",
"Distances",
"Plots",
"StatsPlots",
"StatsBase",
"Statistics",
"Mmap",
"MultivariateStats",
"PyCall",
"Random",
"Primes",
"SparseArrays",
"SHA",
"https://github.com/cjprybol/Mycelia.git#master",
"GenomicAnnotations",
"BioFetch",
"Combinatorics",
"StaticArrays",
"BioSymbols",
"RollingFunctions",
"OrderedCollections",
"Downloads",
"Clustering"
]

for pkg in pkgs
    try
        eval(Meta.parse("import $pkg"))
    catch
        try
            Pkg.add(pkg)
        catch
            Pkg.add(url=pkg)
            pkg = replace(basename(pkg), ".git#master" => "")
        end
        eval(Meta.parse("import $pkg"))
    end
end

    Updating registry at `~/.julia/registries/General`
    Updating git-repo `https://github.com/cjprybol/Mycelia.git#master`
    Updating `~/git/Mycelia/docs/Project.toml`
  [453d265d] ~ Mycelia v0.1.0 `https://github.com/cjprybol/Mycelia.git#master#master` ⇒ v0.1.0 `https://github.com/cjprybol/Mycelia.git#master#master`
    Updating `~/git/Mycelia/docs/Manifest.toml`
  [453d265d] ~ Mycelia v0.1.0 `https://github.com/cjprybol/Mycelia.git#master#master` ⇒ v0.1.0 `https://github.com/cjprybol/Mycelia.git#master#master`
Precompiling project...
  ✓ Mycelia
  1 dependency successfully precompiled in 32 seconds (260 already precompiled)
    Updating git-repo `https://github.com/cjprybol/Mycelia.git#master`
   Resolving package versions...
  No Changes to `~/git/Mycelia/docs/Project.toml`
  No Changes to `~/git/Mycelia/docs/Manifest.toml`


In [3]:
# https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?&id=$(tax_id)
root_tax_id = 287

287

In [4]:
child_tax_ids = vcat(Mycelia.taxonomic_id_to_children(root_tax_id), root_tax_id)
# child_tax_ids = vcat(child_tax_ids, root_tax_id)

MATCH (n)<-[*]-(n2) WHERE n.tax_id IS NOT NULL AND n.tax_id = "287" RETURN DISTINCT n2.tax_id AS tax_id


383-element Vector{Int64}:
 1089456
 1084724
 1081927
 1078464
 1051005
 1051003
 1051004
 1009714
 1000561
  990330
  990327
  990321
  990333
       ⋮
 1454223
 1454221
 1454219
 1454222
 1454220
 1454218
 1454216
 1454217
 1457392
 1457194
 1093787
     287

In [5]:
refseq_metadata = Mycelia.load_refseq_metadata()

,# assembly_accession,bioproject,biosample,wgs_master,refseq_category
,String,String,String,String,String
1,GCF_000001215.4,PRJNA164,SAMN02803731,,reference genome
2,GCF_000001405.39,PRJNA168,,,reference genome
3,GCF_000001635.27,PRJNA169,,,reference genome
4,GCF_000001735.4,PRJNA116,SAMN03081427,,reference genome
5,GCF_000001905.1,PRJNA70973,SAMN02953622,AAGU00000000.3,representative genome
6,GCF_000001985.1,PRJNA32665,SAMN02953685,ABAR00000000.1,representative genome
7,GCF_000002035.6,PRJNA13922,SAMN06930106,,reference genome
8,GCF_000002075.1,PRJNA209509,SAMN02953658,AASC00000000.3,representative genome
9,GCF_000002235.5,PRJNA13728,SAMN00829422,AAGJ00000000.6,representative genome


In [6]:
show(refseq_metadata[1:1, :], allcols=true)

1×23 DataFrame
 Row │ # assembly_accession  bioproject  biosample     wgs_master  refseq_category   taxid  species_taxid  organism_name            infraspecific_name  isolate  version_status  assembly_level  release_type  genome_rep  seq_rel_date  asm_name                submitter                          gbrs_paired_asm  paired_asm_comp  ftp_path                           excluded_from_refseq  relation_to_type_material  asm_not_live_date 
     │ String                String      String        String      String            Int64  Int64          String                   String              String   String          String          String        String      String        String                  String                             String           String           String                             String                String                     String            
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [7]:
is_full = refseq_metadata[!, "genome_rep"] .== "Full"
not_excluded = refseq_metadata[!, "excluded_from_refseq"] .== ""
assembly_levels = ["Complete Genome"]
# assembly_levels = ["Complete Genome", "Chromosome"]
# assembly_levels = ["Complete Genome", "Chromosome", "Scaffold"]
# assembly_levels = ["Complete Genome", "Chromosome", "Scaffold", "Contig"]
assembly_level_filter = map(x -> x in assembly_levels, refseq_metadata[!, "assembly_level"])
tax_id_filter = map(taxid -> taxid in child_tax_ids, refseq_metadata[!, "taxid"])
full_filter = is_full .& not_excluded .& assembly_level_filter .& tax_id_filter
count(full_filter)

389

In [8]:
refseq_metadata_of_interest = refseq_metadata[full_filter, :]

,# assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid
,String,String,String,String,String,Int64
1,GCF_000006765.1,PRJNA57945,SAMN02603714,,reference genome,208964
2,GCF_000014625.1,PRJNA224116,SAMN02603591,,na,208963
3,GCF_000017205.1,PRJNA224116,SAMN02603435,,na,381754
4,GCF_000026645.1,PRJNA224116,SAMEA1705916,,na,557722
5,GCF_000226155.1,PRJNA224116,SAMN02603849,,na,941193
6,GCF_000271365.1,PRJNA224116,SAMN02603895,,na,1093787
7,GCF_000271985.2,PRJNA224116,SAMN02470939,,na,1193501
8,GCF_000284555.1,PRJNA224116,SAMD00061003,,na,1089456
9,GCF_000359505.1,PRJNA224116,SAMN02603669,,na,1280938


In [9]:
for extension in ("genomic.fna.gz", "genomic.gff.gz", "protein.faa.gz")
    outdir = mkpath(joinpath(DIR, extension))
    ProgressMeter.@showprogress for row in DataFrames.eachrow(refseq_metadata_of_interest)
        url = Mycelia.ncbi_ftp_path_to_url(row["ftp_path"], extension)
        outfile = joinpath(outdir, basename(url))
        if !isfile(outfile)
            Downloads.download(url, outfile)
        end
    end
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


In [10]:
extension = "protein.faa.gz"
outdir = mkpath(joinpath(DIR, extension))

"/home/jupyter-cameron.prybol/workspace/2022-02-25-pseudomonas-phylogeny-AA/protein.faa.gz"

In [11]:
# these are too small, all of the within vs between have some disagreement
dna_k = 5
aa_k = 2
# should use these?
# dna_k = 7
# aa_k = 3

2

In [12]:
fastx_files = filter(x -> !occursin(".ipynb_checkpoints", x), readdir(outdir, join=true))

389-element Vector{String}:
 "/home/jupyter-cameron.prybol/workspace/2022-02-25-pseudomonas-phylogeny-AA/protein.faa.gz/GCF_000006765.1_ASM676v1_protein.faa.gz"
 "/home/jupyter-cameron.prybol/workspace/2022-02-25-pseudomonas-phylogeny-AA/protein.faa.gz/GCF_000014625.1_ASM1462v1_protein.faa.gz"
 "/home/jupyter-cameron.prybol/workspace/2022-02-25-pseudomonas-phylogeny-AA/protein.faa.gz/GCF_000017205.1_ASM1720v1_protein.faa.gz"
 "/home/jupyter-cameron.prybol/workspace/2022-02-25-pseudomonas-phylogeny-AA/protein.faa.gz/GCF_000026645.1_ASM2664v1_protein.faa.gz"
 "/home/jupyter-cameron.prybol/workspace/2022-02-25-pseudomonas-phylogeny-AA/protein.faa.gz/GCF_000226155.1_ASM22615v1_protein.faa.gz"
 "/home/jupyter-cameron.prybol/workspace/2022-02-25-pseudomonas-phylogeny-AA/protein.faa.gz/GCF_000271365.1_ASM27136v1_protein.faa.gz"
 "/home/jupyter-cameron.prybol/workspace/2022-02-25-pseudomonas-phylogeny-AA/protein.faa.gz/GCF_000271985.2_ASM27198v2_protein.faa.gz"
 "/home/jupyter-cameron.prybol/w

In [13]:
counts_table, outfile = Mycelia.fasta_list_to_counts_table(fasta_list=fastx_files, k=aa_k, alphabet=:AA, outfile="$(outdir).$(aa_k).counts.bin")

/home/jupyter-cameron.prybol/workspace/2022-02-25-pseudomonas-phylogeny-AA/protein.faa.gz.2.counts.bin found, loading into memory


([26353 27519 … 26006 28784; 16045 16593 … 15767 17371; … ; 0 0 … 0 0; 0 0 … 0 0], "/home/jupyter-cameron.prybol/workspace/2022-02-25-pseudomonas-phylogeny-AA/protein.faa.gz.2.counts.bin")

In [14]:
distance_matrix = counts_matrix_to_distance_matrix(counts_table)
# for i1 in 1:size(counts_table, 2)
#     for i2 in i1+1:size(counts_table, 2)
#         a = counts_table[:, i1]
#         b = counts_table[:, i2]
#         sa = sum(a)
#         sb = sum(b)
#         size_dist = 1-(min(sa, sb)/max(sa, sb))
#         cosine_dist = Distances.cosine_dist(a, b)
#         distances = filter(x -> x > 0, (size_dist, cosine_dist))
#         if isempty(distances)
#             dist = 0.0
#         else
#             dist = reduce(*, distances)
#         end
#         distance_matrix[i1, i2] = distance_matrix[i2, i1] = dist
#     end
# end

In [ ]:
distance_matrix

In [ ]:
newick_tree_file = outfile * ".newick"
Mycelia.distance_matrix_to_newick(distance_matrix, basename.(fastx_files), newick_tree_file)